### Importing Libraries

In [1]:
%pip install scikit-learn
%pip install xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.combine import SMOTEENN

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


Note: you may need to restart the kernel to use updated packages.


#### Reading csv

In [36]:
df=pd.read_csv("data/tel_churn.csv")
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,True,False,False,True,True,...,False,False,True,False,True,False,False,False,False,False
1,1,0,56.95,1889.50,0,False,True,True,False,True,...,False,False,False,True,False,False,True,False,False,False
2,2,0,53.85,108.15,1,False,True,True,False,True,...,False,False,False,True,True,False,False,False,False,False
3,3,0,42.30,1840.75,0,False,True,True,False,True,...,True,False,False,False,False,False,False,True,False,False
4,4,0,70.70,151.65,1,True,False,True,False,True,...,False,False,True,False,True,False,False,False,False,False


In [37]:
df=df.drop('Unnamed: 0',axis=1)

In [38]:
x=df.drop('Churn',axis=1)
x

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,True,False,False,True,True,False,True,...,False,False,True,False,True,False,False,False,False,False
1,0,56.95,1889.50,False,True,True,False,True,False,False,...,False,False,False,True,False,False,True,False,False,False
2,0,53.85,108.15,False,True,True,False,True,False,False,...,False,False,False,True,True,False,False,False,False,False
3,0,42.30,1840.75,False,True,True,False,True,False,True,...,True,False,False,False,False,False,False,True,False,False
4,0,70.70,151.65,True,False,True,False,True,False,False,...,False,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,False,True,False,True,False,True,False,...,False,False,False,True,False,True,False,False,False,False
7028,0,103.20,7362.90,True,False,False,True,False,True,False,...,False,True,False,False,False,False,False,False,False,True
7029,0,29.60,346.45,True,False,False,True,False,True,True,...,False,False,True,False,True,False,False,False,False,False
7030,1,74.40,306.60,False,True,False,True,True,False,False,...,False,False,False,True,True,False,False,False,False,False


In [39]:
num_features = df.select_dtypes(include=['number']).columns
print("Numerical columns:", num_features)

Numerical columns: Index(['SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'Churn'], dtype='object')


In [41]:
cat_features = df.select_dtypes(exclude=['number']).columns
print("Categorical columns:", cat_features)

Categorical columns: Index(['gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes',
       'Dependents_No', 'Dependents_Yes', 'PhoneService_No',
       'PhoneService_Yes', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling_No', 'PaperlessBilling_Yes',
 

In [42]:
y=df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

##### Train Test Split

In [44]:
num_features = x.select_dtypes(exclude="object").columns
cat_features = x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [45]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((5625, 50), (1407, 50))

#### Create an Evaluate Function to give all metrics after model Training

In [46]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [47]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.3688
- Mean Absolute Error: 0.2858
- R2 Score: 0.3031
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.3798
- Mean Absolute Error: 0.2952
- R2 Score: 0.2610


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.3916
- Mean Absolute Error: 0.3197
- R2 Score: 0.2140
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.3949
- Mean Absolute Error: 0.3224
- R2 Score: 0.2009


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.3688
- Mean Absolute Error: 0.2858
- R2 Score: 0.3031
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.3797
- Mean Absolute Error: 0.2952
- R2 Score: 0.2612


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.3381
- Mean Absolute Error: 0.2296
- R2 Score: 0.4143
-----------------------

In [48]:
pred_df=pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
pred_df

,Model Name,R2_Score
2,Ridge,0.261182
0,Linear Regression,0.261042
8,AdaBoost Regressor,0.238476
7,CatBoosting Regressor,0.235200
5,Random Forest Regressor,0.221976
1,Lasso,0.200865
6,XGBRegressor,0.190906
3,K-Neighbors Regressor,0.112408
4,Decision Tree,-0.357399


### Random Forest Regressor is a strong candidate due to its better R² on the training set and relatively lower RMSE on the training set, along with reasonably close predictions on the test set.

#### Random Forest Classifier

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [51]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [52]:
y_pred=model_rf.predict(x_test)

In [53]:
model_rf.score(x_test,y_test)

0.7789623312011372

In [54]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86      1033
           1       0.63      0.42      0.50       374

    accuracy                           0.78      1407
   macro avg       0.72      0.66      0.68      1407
weighted avg       0.76      0.78      0.76      1407



###### As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

###### Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

###### Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [55]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)

In [56]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [57]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [58]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [59]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [60]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [61]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9300341296928327
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       521
           1       0.92      0.96      0.94       651

    accuracy                           0.93      1172
   macro avg       0.93      0.93      0.93      1172
weighted avg       0.93      0.93      0.93      1172



In [62]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model = rf_model.fit(x_train, y_train)  # Fit the model to the training data
y_pred_rf = rf_model.predict(x_test)
score_rf = r2_score(y_test, y_pred_rf) * 100
print("Accuracy of the Random Forest model is %.2f" % score_rf)
pred_df_rf = pd.DataFrame({'Actual Value': y_test, 'Predicted Value': y_pred_rf, 'Difference': y_test - y_pred_rf})
print(pred_df_rf.head())  

Accuracy of the Random Forest model is 23.09
      Actual Value  Predicted Value  Difference
2476             0             0.00        0.00
6773             0             0.09       -0.09
6116             1             0.71        0.29
3047             0             0.17       -0.17
4092             0             0.34       -0.34


In [63]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[468  53]
 [ 29 622]]


#### Performing PCA

In [64]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train1)
xr_test_pca = pca.transform(xr_test1)
explained_variance = pca.explained_variance_ratio_

In [65]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [66]:
model.fit(xr_train_pca,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [67]:
yr_predict_pca = model.predict(xr_test_pca)

In [68]:
model_score_r_pca = model.score(xr_test_pca, yr_test1)

In [69]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1, yr_predict_pca))

0.7107508532423208
              precision    recall  f1-score   support

           0       0.71      0.59      0.64       521
           1       0.71      0.81      0.76       651

    accuracy                           0.71      1172
   macro avg       0.71      0.70      0.70      1172
weighted avg       0.71      0.71      0.71      1172



##### With PCA, we couldn't see any better results.

#### Pickling the model

In [41]:
#import pickle

In [42]:
#filename = 'model.sav'

In [43]:
#pickle.dump(model_rf_smote, open(filename, 'wb'))

In [44]:
#load_model = pickle.load(open(filename, 'rb'))

In [45]:
#model_score_r1 = load_model.score(xr_test1, yr_test1)

In [46]:
#model_score_r1

##### Our final model i.e. RF Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.